In [5]:
# !pip install pyspark
# !pip install -U -q PyDrive
# !apt install openjdk-8-jdk-headless -qq

In [2]:
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [4]:
import re
import sys
import itertools as it
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [2]:
file_path = "./" # Your path

In [8]:
"""
Read and write the each data file.
"""

with open(file_path + 'browsing.txt', 'r') as browsing:
  lines = browsing.readlines()

with open(file_path + 'browsingBasket.txt', 'w') as bbrowsing:
  for (num, line) in enumerate(lines):
    bbrowsing.write("%d:%s" % (num, line))

In [6]:
# conf = SparkConf()
# sc = SparkContext(conf = conf)
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

/home/ljj0512/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [35]:
basklines = sc.textFile(file_path + 'browsingBasket.txt')
basket_Items = basklines.map(lambda l: l.split(':'))
print(basket_Items.take(1))
# output ['0', 'FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 ']
print(basklines.count())
# output 31101

[['0', 'FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 ']]
31101


In [37]:
basketItems = basket_Items.map(lambda p: (int(p[0]), sorted(p[1].split())))
print(basketItems.take(1))
# output (0, ['ELE17451', 'ELE89019', 'FRO11987', 'GRO99222', 'SNA90258'])
print(basketItems.count())
# output 31101 <--- 총 basket 수

[(0, ['ELE17451', 'ELE89019', 'FRO11987', 'GRO99222', 'SNA90258'])]
31101


In [38]:
def itemsOf_baskets(basket):
  """
  Splitting baskets and items.

  Return items in itemList.
  """
  basketId = basket[0]
  itemList = basket[1]
  return [(item, 1) for item in itemList]

In [85]:
items = basketItems.map(itemsOf_baskets)
print(items.take(1))

[[('ELE17451', 1), ('ELE89019', 1), ('FRO11987', 1), ('GRO99222', 1), ('SNA90258', 1)]]


In [39]:
items = basketItems.map(itemsOf_baskets).flatMap(lambda x: x)
print(items.take(1))
# output ('ELE17451', 1)
print(items.count())
# 380824

[('ELE17451', 1)]
380824


In [40]:
supportThresh = 100

In [41]:
itemCounts = items.reduceByKey(lambda c1, c2: c1 + c2)
print(itemCounts.count())
#output 12592

12592


In [56]:
frequent_items = itemCounts.filter(lambda x: x[1] >= supportThresh)  # Filter by expression contained in lambda
print(frequent_items.take(1))
print(frequent_items.count())
#output 647

[('FRO11987', 104)]
647


In [24]:
freqItems = frequent_items.collectAsMap()
print(type(freqItems))
print(len(freqItems)) #output 647
print( next(iter(zip(freqItems.keys(), freqItems.values()))) ) 

<class 'dict'>
647
('FRO11987', 104)


In [25]:
def frequentSingletons(basket):
  """
  Return an array containing the frequency at which the items appear. (Refer to function as itemsOf_baskets -> not include combinations methods)

  This function considers all frequencies.
  """
  basketId = basket[0]
  itemList = basket[1]
  freqList = []

  for item in itemList:
    if item in freqItems:
      freqList.append(item) # 각 usr의 item 중 freq한 itme 추출
  return (basketId, sorted(freqList))

In [43]:
basket_freqSingletons = basketItems.map(frequentSingletons)
print(basket_freqSingletons.take(1))
# output (0, ['ELE17451', 'FRO11987', 'GRO99222', 'SNA90258'])
print(basket_freqSingletons.count())
#output 31101

[(0, ['ELE17451', 'FRO11987', 'GRO99222', 'SNA90258'])]
31101


## mark01

In [57]:
# suport threshold == 100
# frequent sigletons을 하나라도 가지고 있는 basket만을 filter
basket_freqSingletons_count = basket_freqSingletons.filter(lambda x: len(x[1])!=0)
print(basket_freqSingletons_count.count())
#output 30879

30879


In [44]:
def item_pairs_of_basket(basket):
  """
  Compare with above "itemsOf_baskets" function. -> use combinations methods
  """
  basketId = basket[0]
  itemList = basket[1]
  return [(pair_of_items, 1) for pair_of_items in it.combinations(itemList, 2)] 

In [45]:
candidate_doubles = basket_freqSingletons.map(item_pairs_of_basket).flatMap(lambda x: x)
print(candidate_doubles.take(7))
# output (('ELE17451', 'FRO11987'), 1)
print(candidate_doubles.count())
#output 1358611

[(('ELE17451', 'FRO11987'), 1), (('ELE17451', 'GRO99222'), 1), (('ELE17451', 'SNA90258'), 1), (('FRO11987', 'GRO99222'), 1), (('FRO11987', 'SNA90258'), 1), (('GRO99222', 'SNA90258'), 1), (('ELE17451', 'ELE26917'), 1)]
1358611


In [46]:
doubleCounts = candidate_doubles.reduceByKey(lambda c1, c2: c1 + c2)
print(doubleCounts.take(1))
#output (('ELE17451', 'GRO99222'), 148)
print(doubleCounts.count())
#output 149099

[(('ELE17451', 'GRO99222'), 148)]
149099


In [47]:
frequent_pairs = doubleCounts.filter(lambda x: x[1] >= supportThresh)
print(frequent_items.count())

647


In [58]:
freqPairs = frequent_pairs.collectAsMap()
print(type(freqPairs))
print(len(freqPairs)) #output 647
print( next(iter(zip(freqPairs.keys(), freqPairs.values()))) ) 

<class 'dict'>
1334
(('ELE17451', 'GRO99222'), 148)


In [80]:
import itertools as it
def frequentDoubletons(basket_freqSingles):
  """
  Compared with SingleTons functions
  """
  basketId = basket_freqSingles[0]
  itemList = basket_freqSingles[1]
  freqList = []

  for item in it.combinations(itemList,2):
    if item in freqPairs:
      freqList.append(item) # 각 usr의 item 중 freq한 itme 추출
  return (basketId, sorted(freqList))

In [81]:
print(basket_freqSingletons.take(1))

[(0, ['ELE17451', 'FRO11987', 'GRO99222', 'SNA90258'])]


In [84]:
basket_freqDoubletons = basket_freqSingletons.map(frequentDoubletons)
print(basket_freqDoubletons.take(1))
#output (0, ['ELE17451', 'FRO11987', 'GRO99222', 'SNA90258'])
print(basket_freqDoubletons.count())
#output 31101

basket_freqDoubletons_count = basket_freqDoubletons.filter(lambda x: len(x[1])!=0)
print(basket_freqDoubletons_count.count())

[(0, [('ELE17451', 'GRO99222'), ('ELE17451', 'SNA90258'), ('GRO99222', 'SNA90258')])]
31101
24884


In [86]:
def item_triples_of_basket(basket_freqPairs):
	basketId   = basket_freqPairs[0]
	freq_pairs = basket_freqPairs[1]
	freq_items = []
	for p in freq_pairs:
		i1, i2 = p
		if i1 not in freq_items:
			freq_items.append(i1)
		if i2 not in freq_items:
			freq_items.append(i2)
	return 	[ (pair_itmes,1) for pair_itmes in it.combinations(freq_items,3) ]

In [87]:
candidate_triples = basket_freqDoubletons.map(item_triples_of_basket).flatMap(lambda x: x)
print(candidate_triples.take(1))

[(('ELE17451', 'GRO99222', 'SNA90258'), 1)]


In [89]:
tripleCounts = candidate_triples.reduceByKey(lambda c1, c2: c1 + c2)
print(tripleCounts.count()) #output 558176
frequent_triples = tripleCounts.filter(lambda x: x[1] >= supportThresh)
print(frequent_triples.count()) #output 140

558176
140


In [90]:
freqTriples = frequent_triples.collectAsMap()
print(type(freqTriples))
print(len(freqTriples)) #output 647
print( next(iter(zip(freqTriples.keys(), freqTriples.values()))) )

<class 'dict'>
140
(('DAI62779', 'ELE17451', 'FRO78087'), 119)


In [91]:
def rules_fromPairs_sortByConf(pairs, items):
  """
  Confidence, Lift, Conviction
  """
  ruleList = []
  for pair in pairs:
    x = pair[0]
    y = pair[1]

    X_support = items[x]
    Y_support = items[y]
    XYsupport = pairs[pair]
    Conf_XtoY = (1.0 * XYsupport) / X_support
    Conf_YtoX = (1.0 * XYsupport) / Y_support
    ruleList.append((Conf_XtoY, (x, y)))
    ruleList.append((Conf_YtoX, (y, x)))
  return sorted(ruleList, key = lambda x: (-x[0], x[1]))

In [92]:
one2oneRules = rules_fromPairs_sortByConf(freqPairs, freqItems)

In [93]:
topN = 5

In [94]:
for i in range(topN):
  print("Conf: %.8f\tRule: %s => %s" % (one2oneRules[i][0], one2oneRules[i][1][0], one2oneRules[i][1][1]))

print("\n\n")

Conf: 1.00000000	Rule: DAI93865 => FRO40251
Conf: 0.99917628	Rule: GRO85051 => FRO40251
Conf: 0.99065421	Rule: GRO38636 => FRO40251
Conf: 0.99056604	Rule: ELE12951 => FRO40251
Conf: 0.98672566	Rule: DAI88079 => FRO40251





In [127]:
def rules_fromTriples_sortByConf(triples, pairs):
  """
  Confidence, Lift, Conviction
  """
  ruleList = []
  for triple in triples:
    x = triple[0]
    y = triple[1]
    z = triple[2]

    try:
      XY_support = pairs[x,y]
    except KeyError:
      XY_support = pairs[y,x]

    try:
      XZ_support = pairs[x,z]
    except KeyError:
      XZ_support = pairs[z,x]

    try:
      YZ_support = pairs[y,z]
    except KeyError:
      YZ_support = pairs[z,y]

    XYZ_support = triples[triple]

    Conf_XYtoZ = (1.0 * XYZ_support) / XY_support
    Conf_XZtoY = (1.0 * XYZ_support) / XZ_support
    Conf_YZtoX = (1.0 * XYZ_support) / YZ_support
    
    ruleList.append((Conf_XYtoZ, ((x,y),z)))
    ruleList.append((Conf_XZtoY, ((x,z),y)))
    ruleList.append((Conf_YZtoX, ((y,z),x)))
  return sorted(ruleList, key=lambda x: (-x[0],x[1]))


In [128]:
two2oneRules = rules_fromTriples_sortByConf(freqTriples, freqPairs)
print(type(two2oneRules))
print(len(two2oneRules))

<class 'list'>
420


In [130]:
for i in range(topN):
  print("Conf: %.8f\tRule: (%s, %s) => %s" % (two2oneRules[i][0], two2oneRules[i][1][0][0], two2oneRules[i][1][0][1], two2oneRules[i][1][1]))

Conf: 0.98429319	Rule: (DAI62779, GRO85051) => FRO40251
Conf: 0.96280992	Rule: (DAI62779, FRO92469) => FRO40251
Conf: 0.95918367	Rule: (GRO73461, GRO85051) => FRO40251
Conf: 0.95714286	Rule: (ELE92920, GRO81087) => DAI62779
Conf: 0.94945055	Rule: (ELE92920, SNA18336) => DAI62779


In [131]:
def pOne2OneRules2file(rList, N, fname):
  fp = open(fname, 'w+')
  for i in range(N):
    fp.write("Conf: %8f\tRule: %s => %s" % (rList[i][0], rList[i][1][0], rList[i][1][1]))
  fp.close()

def pTwo2OneRules2file(rList, N, fname):
  fp = open(fname, 'w+')
  for i in range(N):
    fp.write("Conf: %.8f\tRule: (%s, %s) => %s" % (rList[i][0], rList[i][1][0][0], rList[i][1][0][1], rList[i][1][1]))

In [132]:
pOne2OneRules2file(one2oneRules, len(one2oneRules), '1-to-1-rules.txt')
pTwo2OneRules2file(two2oneRules, len(two2oneRules), '2-to-1-rules.txt')

In [39]:
sc.stop()